# Libraries

**Elasticsearch Query**

In [1]:
import sys
pathModulesES = '../sauceforyall/'
sys.path.append(pathModulesES)

In [2]:
from yelpquery import YelpQuery
from pandasticsearch import Select
ye = YelpQuery()

**Machine Learning**

In [3]:
import pandas as pd
import numpy as np
import nltk
import pickle
from scipy import sparse

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

**Visualize**

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

**Others**

In [6]:
import re
import string

**Index name**

In [7]:
index_business = "yelp-business*"
index_review = "yelp-review*"
index_tip = "yelp-tip*"
index_user = "yelp-user*"

In [8]:
data_path = "/home/hongphuc95/notebookteam/dataset/"

# 1. Data Pre-Processing

## 1.1 Load and clean data

In [9]:
business_df = pd.read_json(data_path + "business.json", lines=True)

In [10]:
business_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,85016,33.522143,-112.018481,3.0,5,0,{'GoodForKids': 'False'},"Golf, Active Life",None
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"{'RestaurantsReservations': 'True', 'GoodForMe...","Specialty Food, Restaurants, Dim Sum, Imported...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-..."
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,"15655 W Roosevelt St, Ste 237",Goodyear,AZ,85338,33.455613,-112.395596,5.0,3,1,None,"Insurance, Financial Services","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,"4209 Stuart Andrew Blvd, Ste F",Charlotte,NC,28217,35.190012,-80.887223,4.0,4,1,"{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...","Plumbing, Shopping, Local Services, Home Servi...","{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192604,nqb4kWcOwp8bFxzfvaDpZQ,Sanderson Plumbing,,North Las Vegas,NV,89032,36.213732,-115.177059,5.0,9,1,{'BusinessAcceptsCreditCards': 'True'},"Water Purification Services, Water Heater Inst...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
192605,vY2nLU5K20Pee-FdG0br1g,Chapters,17440 Yonge Street,Newmarket,ON,L3Y 6Y9,44.052658,-79.481850,4.5,3,1,"{'RestaurantsPriceRange2': '2', 'BikeParking':...","Books, Mags, Music & Video, Shopping",None
192606,MiEyUDKTjeci5TMfxVZPpg,Phoenix Pavers,21230 N 22nd St,Phoenix,AZ,85024,33.679992,-112.035569,4.5,14,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Home Services, Contractors, Landscaping, Mason...","{'Monday': '7:0-15:0', 'Tuesday': '7:0-15:0', ..."
192607,zNMupayB2jEHVDOji8sxoQ,Beasley's Barber Shop,4406 E Main St,Mesa,AZ,85205,33.416137,-111.735743,4.5,15,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Beauty & Spas, Barbers","{'Tuesday': '8:30-17:30', 'Wednesday': '8:30-1..."


In [11]:
business_df[business_df["business_id"] == "bAa0ue-BJychrcJZd-xSzw"]

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
142616,bAa0ue-BJychrcJZd-xSzw,Lake Mead National Recreation Area,,Boulder City,NV,89005,35.978227,-114.834494,4.0,130,1,"{'BikeParking': 'True', 'GoodForKids': 'True',...","Lakes, Parks, Active Life, Hiking",None


In [12]:
review_df = pd.read_json(data_path + "cleaned/review_cleaned_2016_2019_Vegas.json", lines=True)

In [13]:
review_df

,cool,useful,funny,business_id,stars,review_id,user_id,date,text
0,0,1,0,drLbXX7aRs7_1QAIVfvM_Q,5,J4wxOUA3b54jNlZiZs9HIw,JsueP9ZU6GmtFKAyzdxdlQ,2016-10-17 01:41:37,A lot of guy have already brought up Lawrence....
1,0,0,0,A4zLP5AyKEEHQr_dWEZKig,5,lHfnZJliQ8ptQCeKebOFKA,8cVLhBhpiQx29LQgNe09xw,2016-12-22 17:01:35,Awesome service amaze vegan option with staff ...
2,0,0,0,bcJmbFHVAkLFqhJbUTdbyQ,5,Euu1QJY9rTHF7nLUKBHchQ,lIXDiXM-gh1F1L4O-HxkQQ,2016-04-26 02:39:18,My little daughter love the cream cheese bun f...
3,0,0,0,lY5KRmX58HYs2mGaRBDd2A,2,iULbaheDHBOLZznDsezJ3g,EC5nxNCWCmjHg1F14WrlxQ,2016-11-25 22:58:14,I go here base on the rave review and be shock...
4,0,0,0,8BPmgCjkKDeUdb19w2Hkng,1,myN9QuIWy00q0EY6EdSr4g,EPUdR9EeKwWC9-CEBJ7mBQ,2016-03-20 19:26:25,I contact Chris in regard to a hinge repair I ...
...,...,...,...,...,...,...,...,...,...
1041550,0,0,0,94c68u2I7X6YK54hrjy7FQ,5,cdklGt9GMHYgm0UFpikhUw,yX9kGZKAupFasVtnFvy13g,2018-11-09 07:19:34,Keny be here First off the owner of this place...
1041551,0,0,0,3kdSl5mo9dWC4clrQjEDGg,5,yZbleOxC4_k8n4FclEBIRw,pSvD8Djf3g8eR9VmgaRKhA,2018-11-11 19:11:03,"The service be quick, the food be very good, a..."
1041552,0,0,0,a192hdM0_UVCYLwPJv1Qwg,5,zwzO3yPdGbdgGy3-XQ0SWg,PFiIECX8wuvi7P-1mCvIjA,2018-11-13 21:15:20,"Off the grid Mexican in Vegas. Very tasty, qua..."
1041553,1,1,1,kOo4ZY2UQAX4j312mzQ8mA,5,o8Nc2BJhKJXM6tRHJDRuPA,wu5R5_N7q5iqCBh6NBX26Q,2018-08-13 18:09:16,We hire Taco Naco to cater our family party af...


In [14]:
review_df.shape

(1041555, 9)

In [15]:
#df = pd.read_json(data_path + "cleaned/restaurant_review_cleaned_2018_AZ.json", lines=True)

**Missing data**

In [16]:
missing_cat_business = business_df[business_df.isnull()]["business_id"].tolist()
business_df = business_df.dropna(subset=["categories"])

**To avoid coldstart problem, we filter out all users have less than 10 reviews**

In [17]:
user_review_df_count = review_df.groupby(["user_id", "business_id"]).size().groupby("user_id").size()
user_review_active_df = user_review_df_count[user_review_df_count > 10].reset_index()[["user_id"]]

In [18]:
user_active_df = review_df[review_df["user_id"].isin(user_review_active_df["user_id"])]

In [19]:
#user_active_df = user_active_df[~user_active_df["business_id"].isin(missing_cat_business)]

In [20]:
user_active_df.shape

(288901, 9)

In [21]:
import ast

def fusion_attr(key, val):
    attr_full = ""
    key = key.replace("'","")
    sub_string = re.search('u\'(\w*)\'', val)
    if sub_string:
        attr_full = key + "_" + sub_string.group(1)
    else:
        val = val.replace("'","")
        attr_full = key + "_" + val
    return attr_full
    
def flatten_attr(nested_field):
    l=[]
    if nested_field:
        for (idx,val) in nested_field.items():
            f_level = idx
            s_level = re.findall(r"{(.+)}", val)    
            if s_level:
                for element in s_level[0].split(", "):
                    element_splited = element.split(":")
                    key_element = f_level + "_" + element_splited[0].strip()
                    val_element = element_splited[1].strip()
                    to_add_string = fusion_attr(key_element, val_element)
                    l.append(to_add_string)
            else:
                to_add_string = fusion_attr(f_level, val)
                l.append(to_add_string)    
    return ", ".join(l)

In [22]:
business_df['attributes'] = business_df['attributes'].apply(lambda x: flatten_attr(x))

/home/hongphuc95/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [23]:
review_by_business = user_active_df.groupby('business_id')['text'].agg(lambda col: ' '.join(col)).reset_index()

**Mean rating**

In [24]:
user_active_df = user_active_df.groupby(["user_id", "business_id"], as_index=False).mean()

In [25]:
#user_active_df.to_json("/home/hongphuc95/notebookteam/dataset/cleaned/user_active.json", orient='records', lines=True)

In [26]:
user_active_df.shape

(276641, 6)

**Get relevant data for the recommandation**

In [27]:
#relevant_feature = ["business_id", "user_id", "stars", "text"]
#recommender_df = df[relevant_feature]

In [28]:
#recommender_df.hist(bins=20)

In [29]:
#review_count_df = recommender_df.groupby("user_id")["stars"].count()

In [30]:
#review_count_df[review_count_df >= 10].reset_index()[["user_id"]]

In [31]:
#review_count_df.describe()

In [32]:
#review_count_df.hist(bins=10)

**Filter active user whose has more than 10 reviews**

In [33]:
#filter_active_user = []
#for user in review_count_df[review_count_df >= 10].index:
#    filter_active_user.append(user)

In [34]:
#recommender_df = recommender_df[recommender_df["user_id"].isin(filter_active_user)]

In [35]:
#Take the mean of multiple visits per user
#active_user_df = recommender_df.groupby(["user_id", "business_id"], as_index=False).mean()

In [36]:
#user_id = list(set(active_user_df['user_id']))
#business_id = list(set(active_user_df['business_id']))

In [37]:
#def create_user_id_matrix(row, user_id):
#    return user_id.index(row["user_id"])

#def create_business_id_matrix(row, business_id):
#    return business_id.index(row["business_id"])

In [38]:
#active_user_df["user_id_matrix"] = active_user_df.apply(lambda x: create_user_id_matrix(x, user_id), axis = 1)

In [39]:
#active_user_df["business_id_matrix"] = active_user_df.apply(lambda x: create_business_id_matrix(x, business_id), axis = 1)

# 1.2 Before going in

In [24]:
business_df.head(5)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,85016,33.522143,-112.018481,3.0,5,0,GoodForKids_False,"Golf, Active Life",None
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"RestaurantsReservations_True, GoodForMeal_dess...","Specialty Food, Restaurants, Dim Sum, Imported...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"GoodForKids_True, NoiseLevel_average, Restaura...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-..."
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,"15655 W Roosevelt St, Ste 237",Goodyear,AZ,85338,33.455613,-112.395596,5.0,3,1,,"Insurance, Financial Services","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,"4209 Stuart Andrew Blvd, Ste F",Charlotte,NC,28217,35.190012,-80.887223,4.0,4,1,"BusinessAcceptsBitcoin_False, ByAppointmentOnl...","Plumbing, Shopping, Local Services, Home Servi...","{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ..."


In [25]:
business_df.shape

(192127, 14)

In [26]:
user_active_df.head(5)

,cool,useful,funny,business_id,stars,review_id,user_id,date,text
2,0,0,0,bcJmbFHVAkLFqhJbUTdbyQ,5,Euu1QJY9rTHF7nLUKBHchQ,lIXDiXM-gh1F1L4O-HxkQQ,2016-04-26 02:39:18,My little daughter love the cream cheese bun f...
3,0,0,0,lY5KRmX58HYs2mGaRBDd2A,2,iULbaheDHBOLZznDsezJ3g,EC5nxNCWCmjHg1F14WrlxQ,2016-11-25 22:58:14,I go here base on the rave review and be shock...
5,0,1,0,PtET3ArSSmvwlA-lt1vNZQ,4,wx19j82Uz70zIr8h43ioLg,fbbXmHoCfd6wXMHnkjgDTg,2016-04-14 18:19:00,I go in on a whim today because wow do I need ...
13,4,4,3,zuwba6QEBIDZT0tJZmNhdQ,1,mkTV-8wnjQQYXI9jjnR6_g,AsYBFmnMZMm9FOJi6o0seA,2016-07-06 03:55:27,I have all you can eat sush. Quality of fish b...
16,0,0,0,WRQ-9LluyivReFiQZFUujw,5,3o8wogOG4rgs9LhQvVfCPw,fkPTEFUGmIGE_ts48PZoIA,2016-05-22 00:19:41,You be miss out if you have not try this place...


# Content Based

## Based on user review

In [12]:
review_by_business = user_active_df.groupby('business_id')['text'].agg(lambda col: ' '.join(col)).reset_index()

NameError: name 'user_active_df' is not defined

In [28]:
from nltk.tokenize import word_tokenize

In [29]:
import gensim 
from gensim.models import Word2Vec

In [30]:
review_by_business["text_tokenize"] = review_by_business["text"].apply(lambda x: word_tokenize(x.lower()))

In [32]:
review_by_business

,business_id,text,text_tokenize
0,--9e1ONYQuAa-CB_Rrw7Tw,The service be perfect Our glass be never even...,"[the, service, be, perfect, our, glass, be, ne..."
1,--WsruI0IGEoeRmkErU5Gg,Again great service I have my up stair carpet ...,"[again, great, service, i, have, my, up, stair..."
2,--Y7NhBKzLTbNliMUX_wfg,These guy be great BAR NONE. I be somewhat fam...,"[these, guy, be, great, bar, none, ., i, be, s..."
3,--e8PjCNhEz32pprnPhCwQ,"Do not take your cat here, they will charge yo...","[do, not, take, your, cat, here, ,, they, will..."
4,--z7PM8AGaJP0aBmGMY7RA,Nacho be the most reliable and honest mechanic...,"[nacho, be, the, most, reliable, and, honest, ..."
...,...,...,...
19622,zyrF7je6qVbaJMPfMG75xg,"I love the practitioner here. The nurse, front...","[i, love, the, practitioner, here, ., the, nur..."
19623,zyvrQstmpEKIC_KQvK7LGw,Used Grencare in the past to build our pool at...,"[used, grencare, in, the, past, to, build, our..."
19624,zzO9QVUj-XvZ8trNX2lqAg,I be now a Nevada resident come from Californi...,"[i, be, now, a, nevada, resident, come, from, ..."
19625,zzOo9n22fBbKAhbSpMzggA,we be Locals that live across the Road from Go...,"[we, be, locals, that, live, across, the, road..."


In [33]:
model = gensim.models.Word2Vec(review_by_business["text_tokenize"], min_count = 1, size = 100, workers = 4) 

In [34]:
review_by_business["result"] = review_by_business["text_tokenize"].apply(lambda x: model.wv[x])

In [37]:
review_by_business.drop(columns=["text", "text_tokenize"], axis=1, inplace=True)

In [47]:
model.most_similar(positive=['fresh', 'food'], topn = 10)

/home/hongphuc95/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('freshly', 0.5744677186012268),
 ('ingredients', 0.5674877166748047),
 ('ingredient', 0.5672014355659485),
 ('consistently', 0.5640529990196228),
 ('authentic', 0.5388814806938171),
 ('pip', 0.538441002368927),
 ('hummus', 0.5376486778259277),
 ('freshness', 0.5299689769744873),
 ('food-', 0.5295577645301819),
 ('pizza', 0.519170880317688)]

In [39]:
# Calculate cosine similarity between two vectors 
def cossim(v1, v2): 
    return np.dot(v1, v2) / np.sqrt(np.dot(v1, v1)) / np.sqrt(np.dot(v2, v2)) 

In [40]:
def keyword_recommend(input_str, docvecs):
    from nltk.tokenize import word_tokenize
    input_vec = pd.DataFrame({"business_id" : ["1"], "text" : [input_str]})
    input_vec["text_tokenize"] = input_vec["text"].apply(lambda x: word_tokenize(x))

    input_vec["result"] = input_vec["text_tokenize"].apply(lambda x: model.wv[x])
    input_vec = input_vec["result"]
    docvecs = docvecs[["business_id", "result"]]

    docvecs["similarity"] = docvecs["result"].apply(lambda x: cossim(x[0] , input_vec.values[0][0]))
    
    res = pd.DataFrame()
    res = docvecs.drop(columns=["result"])
    return res.sort_values(ascending=False, by="similarity")

In [49]:
hm = keyword_recommend("tacos", review_by_business)

In [50]:
hm.head(20)

,business_id,similarity
4195,CiYLq33nAyghFkUR15pP-Q,1.000000
1167,2vr2yGIzSehe_ITFamNpyw,1.000000
15892,nsOdZH6msYhXOsJEBQdq4w,0.676851
6985,LnCNoBfyLQVdk8CLj6ra4w,0.676851
3916,BqXuk5peVO-lIovXCv9CyQ,0.674584
16631,qB0WAMveyK2-3itLPOiqVw,0.666398
18090,v7xjDbezQ2xw95itvN244Q,0.664913
16107,oaap_zKAxkrlTBwF3onmXw,0.623254
4311,D74O-i-iEWo-e5BpwFMcUA,0.597881
12173,bUEc1ZDgfZPymHyUXEiiNg,0.597881


In [49]:
item_ids = review_by_business['business_id'].tolist()

In [50]:
from sklearn.preprocessing import normalize
def get_item_profile(item_id):
    idx = item_ids.index(item_id)
    item_profile = tfidf_matrix[idx:idx+1]
    return item_profile

def get_item_profiles(ids):
    item_profiles_list = [get_item_profile(x) for x in ids]
    item_profiles = sparse.vstack(item_profiles_list)
    return item_profiles

def build_users_profile(user_id, user_active_indexed_df):
    user_behaviour_df = user_active_indexed_df.loc[user_id]
    user_item_profiles = get_item_profiles(user_behaviour_df['business_id'])
    user_item_score = np.array(user_behaviour_df['stars']).reshape(-1,1)

    #Weighted average of item profiles by the interactions strength
    user_item_score_weighted_avg = np.sum(user_item_profiles.multiply(user_item_score), axis=0) / np.sum(user_item_score)
    user_profile_norm = normalize(user_item_score_weighted_avg)
    return user_profile_norm

def build_users_profiles(): 
                                        
    user_active_indexed_df = user_active_df[user_active_df["business_id"].isin(review_by_business["business_id"])].set_index("user_id")    
    user_profiles = {}
    for user_id in user_active_indexed_df.index.unique():
        user_profiles[user_id] = build_users_profile(user_id, user_active_indexed_df)
    return user_profiles

In [65]:
user_profiles = build_users_profiles()

In [81]:
test_user = user_profiles["xNGBY48SkzWcwSd88CYoOw"]

In [83]:
pd.DataFrame(sorted(zip(tfidf_feature_names, 
                        user_profiles["xNGBY48SkzWcwSd88CYoOw"].flatten().tolist()), key=lambda x: -x[1])[:20],
             columns=['token', 'relevance'])

,token,relevance
0,place,0.217634
1,good,0.187108
2,food,0.183061
3,great,0.167971
4,time,0.149359
5,like,0.140584
6,brisket,0.138920
7,love,0.133528
8,try,0.131400
9,just,0.121892


In [72]:
class ContentBasedRecommender:
    
    MODEL_NAME = 'Content-Based'
    
    def __init__(self, items_df=None):
        self.item_ids = item_ids
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def _get_similar_items_to_user_profile(self, user_id, topn=1000):
        #Computes the cosine similarity between the user profile and all item profiles
        cosine_similarities = cosine_similarity(user_profiles[user_id], tfidf_matrix)
        #Gets the top similar items
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        #Sort the similar items by similarity
        similar_items = sorted([(item_ids[i], cosine_similarities[0,i]) for i in similar_indices], key=lambda x: -x[1])
        return similar_items
        
    def recommend(self, user_id, items_to_ignore = [], topn=10):
        
        similar_items = self._get_similar_items_to_user_profile(user_id)
        #Ignores items the user has already interacted
        similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        
        recommendations_df = pd.DataFrame(similar_items_filtered, columns=['business_id', 'rec_score']) \
                                    .head(topn)

        return recommendations_df

In [73]:
content_based_rec = ContentBasedRecommender(business_df)

In [79]:
content_based_rec.recommend("xNGBY48SkzWcwSd88CYoOw")

,business_id,rec_score
0,wl0QZqAzr1DelslQ02JGCQ,0.698083
1,ohEnmKpF7i2_ujme1p_vUQ,0.680026
2,8796vXaydw2aY_zER0n9uA,0.668682
3,cTJjTKz2huGZ-ElScC2pSw,0.668353
4,OjApDLQUPV8c_yPCg8xZog,0.665223
5,VyVIneSU7XAWgMBllI6LnQ,0.662720
6,0ldxjei8v4q95fApIei3Lg,0.657522
7,Eg_M8eeu9SDEdbP7iUmVbQ,0.655764
8,A7gK6ZrGy939Zo43xNZcVw,0.652435
9,n8Zqqhff-2cxzWt_nwhU2Q,0.651179


In [80]:
user_active_df[user_active_df["user_id"] == "xNGBY48SkzWcwSd88CYoOw"]

,cool,useful,funny,business_id,stars,review_id,user_id,date,text
49604,1,1,0,8gwTY5nZmV3CiP9xRYc--w,5,f5I9AkQDaB15haWlriw_EQ,xNGBY48SkzWcwSd88CYoOw,2015-03-25 18:05:51+00:00,"I love this store, my favorite place to shop. ..."
75346,0,0,0,huk3l1YLv75Ew9rci3FNcQ,3,DA7acoTDMOjhPXm9FTTgOA,xNGBY48SkzWcwSd88CYoOw,2015-03-25 17:47:46+00:00,"All the cardio equipment have personal TVs, I ..."
333693,0,0,0,Xg5qEQiB-7L6kGJ5F4K3bQ,5,L6QYuM-9x8oUfNrttuBN9g,xNGBY48SkzWcwSd88CYoOw,2015-08-12 18:25:44+00:00,"WOW, this be good bq. I have ate at a lot of b..."
424407,0,0,0,4m6bVRcNJYvaPKi9Tg5MGw,5,H2kdea0ijuUlcEeptq2CDg,xNGBY48SkzWcwSd88CYoOw,2015-05-06 00:17:46+00:00,I be so happy they put a hardware store back o...
445779,1,1,1,wl0QZqAzr1DelslQ02JGCQ,5,AYFU_qGcMSx7IeMg-geYfg,xNGBY48SkzWcwSd88CYoOw,2015-08-12 18:30:52+00:00,"I really enjoy eat here, they keep it really d..."
499997,0,0,0,bKoLvj4R8OUE2tFiqDvv_w,5,9qddh77az4xMpCjZN55jgg,xNGBY48SkzWcwSd88CYoOw,2015-05-20 18:23:57+00:00,"I love True Rest, I have be go here since they..."
540985,0,0,0,whY2KIvO60ZZwRceqzkqyw,4,XhKs-3P7-nhQFkqm2PTSkQ,xNGBY48SkzWcwSd88CYoOw,2015-05-06 00:09:36+00:00,"Nice new small local bar, they do a good job o..."
622348,0,0,0,u9R0_pvEkl7QGPJYH3j1sg,5,fNMnaGGZ6atMIeaYi8GWAg,xNGBY48SkzWcwSd88CYoOw,2015-05-07 17:01:23+00:00,I have not have the pizza here but I have the ...
875879,0,0,0,mGcNhuOzpgc3colTDjI1bw,5,rqVqeHy3hebAzPpHTNIf9A,xNGBY48SkzWcwSd88CYoOw,2015-05-06 00:37:34+00:00,"Ok so I do not go here yet, but while I be at ..."
876208,0,1,0,VyVIneSU7XAWgMBllI6LnQ,4,Pp7djhIIwXyRS50KvZI_nw,xNGBY48SkzWcwSd88CYoOw,2015-03-25 17:45:48+00:00,I love the lunch avocado brisket salad. Althou...


In [93]:
a = review_by_business[review_by_business["business_id"] == "wl0QZqAzr1DelslQ02JGCQ"]

In [95]:
a.to_string()

"                  business_id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [26]:
vectorizer = TfidfVectorizer(analyzer = 'word', stop_words = 'english', 
                             lowercase = True, max_features = 5000,
                             ngram_range = (1, 1)
                            )

In [27]:
tfidf_matrix = vectorizer.fit_transform(business_df["categories"] + business_df["attributes"])
tfidf_feature_names = vectorizer.get_feature_names()

In [30]:
item_ids = business_df['business_id'].tolist()

In [31]:
from sklearn.preprocessing import normalize
def get_item_profile(item_id):
    idx = item_ids.index(item_id)
    item_profile = tfidf_matrix[idx:idx+1]
    return item_profile

def get_item_profiles(ids):
    item_profiles_list = [get_item_profile(x) for x in ids]
    item_profiles = sparse.vstack(item_profiles_list)
    return item_profiles

def build_users_profile(user_id, user_active_indexed_df):
    user_behaviour_df = user_active_indexed_df.loc[user_id]
    user_item_profiles = get_item_profiles(user_behaviour_df['business_id'])
    user_item_score = np.array(user_behaviour_df['stars']).reshape(-1,1)

    #Weighted average of item profiles by the interactions strength
    user_item_score_weighted_avg = np.sum(user_item_profiles.multiply(user_item_score), axis=0) / np.sum(user_item_score)
    user_profile_norm = normalize(user_item_score_weighted_avg)
    return user_profile_norm

def build_users_profiles(): 
                                        
    user_active_indexed_df = user_active_df[user_active_df["business_id"].isin(business_df["business_id"])].set_index("user_id")    
    user_profiles = {}
    for user_id in user_active_indexed_df.index.unique():
        user_profiles[user_id] = build_users_profile(user_id, user_active_indexed_df)
    return user_profiles

In [32]:
user_profiles = build_users_profiles()
len(user_profiles)

29844

In [33]:
user_profiles

{'---1lKK3aKOuomHnwAkAow': array([[0., 0., 0., ..., 0., 0., 0.]]),
 '--2vR0DIsmQ6WfcSzKWigw': array([[0., 0., 0., ..., 0., 0., 0.]]),
 '--Nnm_506G_p8MxAOQna5w': array([[0., 0., 0., ..., 0., 0., 0.]]),
 '--PHyzUR6FEjHqxg9I60LA': array([[0., 0., 0., ..., 0., 0., 0.]]),
 '--Qh8yKWAvIP4V4K8ZPfHA': array([[0., 0., 0., ..., 0., 0., 0.]]),
 '--RY4bG7REOBum5HsSspxQ': array([[0., 0., 0., ..., 0., 0., 0.]]),
 '--RlSfc-QmcHFGHyX6aVjA': array([[0., 0., 0., ..., 0., 0., 0.]]),
 '--YhjyV-ce1nFLYxP49C5A': array([[0., 0., 0., ..., 0., 0., 0.]]),
 '--amZ_cR9Zgu4B1RrAHJyA': array([[0., 0., 0., ..., 0., 0., 0.]]),
 '--fF_pQlaU9sME-HLCoHlQ': array([[0., 0., 0., ..., 0., 0., 0.]]),
 '--ty7Z9fEt08E3dS3_qoSA': array([[0., 0., 0., ..., 0., 0., 0.]]),
 '-0-hVEpwWEcJLJoGq3rE3g': array([[0., 0., 0., ..., 0., 0., 0.]]),
 '-05XqtNjcBq19vh2CVJN8g': array([[0.        , 0.        , 0.06545271, ..., 0.        , 0.        ,
         0.        ]]),
 '-0b84SUGVN0YkG5j2MCmBw': array([[0., 0., 0., ..., 0., 0., 0.]]),
 '-0d

In [34]:
test_user = user_profiles["--2vR0DIsmQ6WfcSzKWigw"]

pd.DataFrame(sorted(zip(tfidf_feature_names, 
                        user_profiles["--2vR0DIsmQ6WfcSzKWigw"].flatten().tolist()), key=lambda x: -x[1])[:20],
             columns=['token', 'relevance'])

,token,relevance
0,hotels,0.404997
1,businessparking_garage_true,0.255184
2,casinos,0.254973
3,wifi_paid,0.250633
4,entertainment,0.228629
5,arts,0.216157
6,businessparking_valet_true,0.210491
7,resorts,0.208821
8,event,0.166525
9,planning,0.165766


In [64]:
class ContentBasedRecommender:
    
    MODEL_NAME = 'Content-Based'
    
    def __init__(self, items_df=None):
        self.item_ids = item_ids
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def _get_similar_items_to_user_profile(self, user_id, topn=1000):
        #Computes the cosine similarity between the user profile and all item profiles
        cosine_similarities = cosine_similarity(user_profiles[user_id], tfidf_matrix)
        #Gets the top similar items
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        #Sort the similar items by similarity
        similar_items = sorted([(item_ids[i], cosine_similarities[0,i]) for i in similar_indices], key=lambda x: -x[1])
        return similar_items
        
    def recommend(self, user_id, topn=10):
        
        reviewed_business_df = user_active_df[user_active_df["user_id"] == user_id]
        business_df_mod = business_df.rename(columns={"stars": "stars_business"})
        reviewed_business_df = pd.merge(reviewed_business_df, business_df_mod, how='left', left_on="business_id", right_on="business_id")
        reviewed_business_df = reviewed_business_df[["business_id", "name", "stars"]]
        already_reviewed = reviewed_business_df["business_id"]
        
        items_to_ignore = already_reviewed.values
        
        similar_items = self._get_similar_items_to_user_profile(user_id)
        #Ignores items the user has already interacted
        similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        
        recommendations_df = pd.DataFrame(similar_items_filtered, columns=['business_id', 'rec_score']) \
                                    .head(topn)
        
        recommendations_df = pd.merge(recommendations_df, business_df, how='left', left_on="business_id", right_on="business_id")
        recommendations_df = recommendations_df[["business_id", "name", "categories", "attributes"]]

        return recommendations_df

In [65]:
content_based_rec = ContentBasedRecommender(business_df)

In [66]:
content_based_rec.recommend("--2vR0DIsmQ6WfcSzKWigw")

,business_id,name,categories,attributes
0,ajoqEHnCZTD8-8GqGLq9-Q,Palms Casino Resort,"Arts & Entertainment, Hotels & Travel, Event P...","RestaurantsReservations_True, OutdoorSeating_T..."
1,Az_60nNuh1FH8Ds8oasZjw,Encore,"Arts & Entertainment, Event Planning & Service...","WiFi_paid, BusinessAcceptsCreditCards_True, Re..."
2,eAc9Vd6loOgRQolMXQt6FA,Mandalay Bay Resort & Casino,"Resorts, Arts & Entertainment, Restaurants, Ho...","RestaurantsReservations_True, RestaurantsDeliv..."
3,GGoQdKo8QgKfTW23hf4Tuw,MGM Grand Poker Room,"Hotels & Travel, Event Planning & Services, Ar...","RestaurantsReservations_False, GoodForKids_Fal..."
4,ORmMxiJnz9sMnHLFTZtitw,Mandalay Bay Sports Book,"Arts & Entertainment, Hotels, Hotels & Travel,...","BusinessAcceptsCreditCards_True, BusinessParki..."
5,cJbihpmefzTASQV4XZR9lQ,ARIA Race & Sports Book,"Hotels, Resorts, Hotels & Travel, Casinos, Art...","BusinessAcceptsCreditCards_True, BusinessParki..."
6,f8IMQgRwo-8GP372MElDGQ,Plaza Hotel & Casino,"Event Planning & Services, Restaurants, Hotels...","NoiseLevel_loud, RestaurantsPriceRange2_2, Out..."
7,xtYiHTmunjfCN2sUaQxBjA,Rio All Suites Hotel & Casino,"Shopping, Arts & Entertainment, Food, Indian, ...","RestaurantsPriceRange2_2, BusinessAcceptsCredi..."
8,OFnDWBP8iXxgeBrfHA5fRA,Four Queens Hotel & Casino,"Hotels & Travel, Arts & Entertainment, Hotels,...","OutdoorSeating_False, RestaurantsGoodForGroups..."
9,5Zc41a446gV3K_o7CDs69Q,Riviera Hotel & Casino,"Hotels, Arts & Entertainment, Hotels & Travel,...","RestaurantsPriceRange2_2, RestaurantsAttire_ca..."


# Collaborative Filtering

In [28]:
users_items_pivot_matrix_df = user_active_df.pivot(index="user_id", columns="business_id", values="stars").fillna(0)

In [30]:
users_items_pivot_matrix_df = user_active_df.pivot(index="user_id", columns="business_id",
                                                           values="stars").fillna(0)

In [31]:
ratings_matusers_items_pivot_matrix = users_items_pivot_matrix_df.values
users_ids = list(users_items_pivot_matrix_df.index)
user_col = users_items_pivot_matrix_df.columns

In [32]:
users_items_pivot_sparse_matrix = sparse.csr_matrix(users_items_pivot_matrix_df.values)
users_items_pivot_sparse_matrix

<11863x19627 sparse matrix of type '<class 'numpy.float64'>'
	with 276641 stored elements in Compressed Sparse Row format>

In [33]:
from scipy.sparse.linalg import svds

In [34]:
#The number of factors to factor the user-item matrix.
NUMBER_OF_FACTORS_MF = 15
#Performs matrix factorization of the original user item matrix
U, sigma, Vt = svds(users_items_pivot_sparse_matrix, k = NUMBER_OF_FACTORS_MF)
sigma = np.diag(sigma)

In [35]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 
all_user_predicted_ratings

array([[ 2.58294220e-02, -2.31549185e-03, -2.40659422e-04, ...,
         3.62515596e-03,  8.65535039e-05,  3.07483195e-03],
       [-2.81668414e-03,  2.21997923e-04,  6.27093789e-04, ...,
         2.21936617e-04, -2.10298890e-05,  2.28870349e-03],
       [ 1.53950923e-01,  2.86391689e-03,  3.60212037e-04, ...,
        -1.81171828e-04, -5.24453805e-06,  7.51055937e-03],
       ...,
       [ 1.15848525e-02,  6.34629538e-05, -4.22764455e-04, ...,
         6.68951424e-04,  1.16575411e-05, -5.34926889e-05],
       [-1.16879453e-02,  5.02041591e-04,  2.17299508e-03, ...,
        -6.21107690e-04,  1.48822037e-05, -1.56511192e-03],
       [ 6.15526882e-03,  3.79248296e-04,  1.35771335e-03, ...,
         5.34117426e-04,  1.62953182e-05,  2.32961496e-03]])

In [36]:
all_user_predicted_ratings_norm = (all_user_predicted_ratings - all_user_predicted_ratings.min()) / (all_user_predicted_ratings.max() - all_user_predicted_ratings.min())

In [37]:
cf_preds_df = pd.DataFrame(all_user_predicted_ratings_norm, columns = users_items_pivot_matrix_df.columns, index=list(users_items_pivot_matrix_df.index)).transpose()
cf_preds_df.head(10)

,--2vR0DIsmQ6WfcSzKWigw,--RlSfc-QmcHFGHyX6aVjA,--ZNfWKj1VyVElRx6-g1fg,-05XqtNjcBq19vh2CVJN8g,-0OyT3le0GqdyvRLLfB7MQ,-16aKjco1c0RJ7c4U-q_Kw,-191gKrqDzXGUrpl7npkXw,-1x8gXJnrI-FeZPNvnfbRg,-2gOxVWcnBr5DclrrsWXCA,-3Tgw9br9O68emgrgFj48Q,...,zwWEDIta87K8LBkt_1KzHA,zwWjcahrW40svr0V3fLr7w,zwo4S612wfakZodLQ8aMIQ,zx5rdBK9NFZrAglIXF1LiQ,zxmIKyEX89aY0QLC1JJksA,zy1CuLKzsy4WTggLOmRv5Q,zyMB50UtmEuQf-1VUKxZxQ,zynbfDYT7y-OqdLGHcYy_g,zz9rIM0FmlWYvAzr6dGwVQ,zzHSF3oCjXqgsYuVJPgdvw
business_id,,,,,,,,,,,,,,,,,,,,,
--9e1ONYQuAa-CB_Rrw7Tw,0.558218,0.557227,0.562654,0.556962,0.558490,0.557153,0.556797,0.557028,0.557866,0.557696,...,0.557075,0.557638,0.556845,0.556548,0.559966,0.559374,0.562259,0.557725,0.556920,0.557537
--WsruI0IGEoeRmkErU5Gg,0.557244,0.557332,0.557424,0.557341,0.557338,0.557321,0.557372,0.557321,0.557319,0.557330,...,0.557321,0.557352,0.557345,0.557324,0.557326,0.557317,0.557358,0.557327,0.557342,0.557338
--Y7NhBKzLTbNliMUX_wfg,0.557316,0.557346,0.557337,0.557390,0.557411,0.557416,0.557353,0.557323,0.557308,0.557321,...,0.557349,0.557330,0.557436,0.557304,0.557326,0.557333,0.557404,0.557310,0.557400,0.557371
--e8PjCNhEz32pprnPhCwQ,0.557325,0.557326,0.557329,0.557333,0.557324,0.557320,0.557330,0.557325,0.557330,0.557325,...,0.557326,0.557325,0.557310,0.557330,0.557329,0.557330,0.557326,0.557325,0.557327,0.557315
--z7PM8AGaJP0aBmGMY7RA,0.557301,0.557321,0.557347,0.557295,0.557362,0.557317,0.557273,0.557317,0.557329,0.557330,...,0.557319,0.557325,0.557295,0.557333,0.557372,0.557374,0.557315,0.557340,0.557340,0.557362
-0BxAGlIk5DJAGVkpqBXxg,0.557699,0.557284,0.557353,0.557298,0.557295,0.557327,0.557373,0.557317,0.557402,0.557329,...,0.557320,0.557295,0.557450,0.557302,0.557418,0.557265,0.557294,0.557380,0.557283,0.557327
-0RkJ_uIduNLWQrphbADRw,0.557056,0.557382,0.557175,0.557859,0.557549,0.557426,0.557175,0.557376,0.556965,0.557497,...,0.557253,0.557645,0.557404,0.556891,0.557695,0.557042,0.558722,0.557567,0.557519,0.557452
-0gEYa7iHAshDcMvJ0Vuag,0.557513,0.557336,0.557462,0.557301,0.557086,0.557216,0.557524,0.557390,0.557555,0.557392,...,0.557311,0.557405,0.557428,0.557457,0.557355,0.557202,0.557104,0.557382,0.557164,0.557247
-0qht1roIqleKiQkBLDkbw,0.558011,0.557170,0.556915,0.557667,0.556964,0.557501,0.557667,0.558435,0.557295,0.557259,...,0.557490,0.557322,0.557289,0.557526,0.556815,0.557194,0.556938,0.557415,0.557389,0.557379


In [57]:
class CFRecommender:
    
    MODEL_NAME = 'Collaborative Filtering'
    
    def __init__(self, predictions_df, items_df=None):
        self.predictions_df = predictions_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend(self, user_id, items_to_ignore=[], topn=30):

        # Get and sort the user's predictions
        sorted_user_predictions = self.predictions_df[user_id].sort_values(ascending=False) \
                                    .reset_index() \
                                    .rename(columns={user_id: 'rec_score'})
        
        
        # Get list of business that user is already voted
        #items_to_ignore = self.items_df[self.items_df['user_id'] == user_id]['business_id']

        reviewed_business_df = user_active_df[user_active_df["user_id"] == user_id]
        business_df_mod = business_df.rename(columns={"stars": "stars_business"})
        reviewed_business_df = pd.merge(reviewed_business_df, business_df_mod, how='left', left_on="business_id", right_on="business_id")
        reviewed_business_df = reviewed_business_df[["business_id", "name", "stars"]]
        already_reviewed = reviewed_business_df["business_id"].values
        items_to_ignore = already_reviewed
        
        # Recommend the highest predicted rating movies that the user hasn't seen yet.
        recommendations_df = sorted_user_predictions[~sorted_user_predictions['business_id'].isin([items_to_ignore])] \
                               .sort_values('rec_score', ascending = False) \
                               .head(topn)
        #recommendations_df = pd.merge(recommendations_df, business_df, how='left', left_on="business_id", right_on="business_id")
        #recommendations_df = recommendations_df[["business_id", "name"]]
        return recommendations_df


In [58]:
cf_recommender_model = CFRecommender(cf_preds_df, user_active_df)

In [59]:
a = cf_recommender_model.recommend("--2vR0DIsmQ6WfcSzKWigw")

In [60]:
a

,business_id,rec_score
0,FaHADZARwnY4yvlvpnsfGA,0.595339
1,na4Th5DrNauOv-c43QQFvA,0.582417
2,Wxxvi3LZbHNIDwJ-ZimtnA,0.581801
3,JAmQCmczUclNUfZjkNdjQA,0.579028
4,VyjyHoBg3KC5BSFRlD0ZPQ,0.578397
5,3xmfT7l3xNH5LK1dLzfvGw,0.576692
6,5LNZ67Yw9RD6nf4_UhXOjw,0.576240
7,NCFwm2-TDb-oBQ2medmYDg,0.575598
8,qjnJFZtsY_nfRzoL3J_UWQ,0.574682
9,t-o_Sraneime4DDhWrQRBA,0.573128


# Hybird

In [71]:
class HybridRecommender:
    
    MODEL_NAME = 'Hybrid'
    
    def __init__(self, cb_rec_model, cf_rec_model, items_df, cb_ensemble_weight=1.0, cf_ensemble_weight=1.0):
        self.cb_rec_model = cb_rec_model
        self.cf_rec_model = cf_rec_model
        self.cb_ensemble_weight = cb_ensemble_weight
        self.cf_ensemble_weight = cf_ensemble_weight
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend(self, user_id, items_to_ignore=[], topn=10):
        #Getting the top-1000 Content-based filtering recommendations
        cb_recs_df = self.cb_rec_model.recommend(user_id, items_to_ignore=items_to_ignore,
                                                           topn=1000).rename(columns={'rec_score': 'rec_score_cb'})
        
        #Getting the top-1000 Collaborative filtering recommendations
        cf_recs_df = self.cf_rec_model.recommend(user_id, items_to_ignore=items_to_ignore, 
                                                           topn=1000).rename(columns={'rec_score': 'rec_score_cf'})
        
        #Combining the results by contentId
        recs_df = cb_recs_df.merge(cf_recs_df,
                                   how = 'outer', 
                                   left_on = 'business_id', 
                                   right_on = 'business_id').fillna(0.0)
        
        #Computing a hybrid recommendation score based on CF and CB scores
        #recs_df['recStrengthHybrid'] = recs_df['recStrengthCB'] * recs_df['recStrengthCF'] 
        recs_df['rec_score_hybrid'] = (recs_df['rec_score_cb'] * self.cb_ensemble_weight) \
                                     + (recs_df['rec_score_cf'] * self.cf_ensemble_weight)
        
        #Sorting recommendations by hybrid score
        recommendations_df = recs_df.sort_values('rec_score_hybrid', ascending=False).head(topn)

        return recommendations_df

In [72]:
hybrid_rec = HybridRecommender(content_based_rec, cf_recommender_model, business_df,
                                             cb_ensemble_weight=1.0, cf_ensemble_weight=100.0)

In [73]:
hybrid_rec.recommend("--2vR0DIsmQ6WfcSzKWigw", topn=20)

,business_id,rec_score_cb,rec_score_cf,rec_score_hybrid
1000,RESDUcs7fIiihp38-d6_6g,0.000000,0.493482,49.348213
1001,NCFwm2-TDb-oBQ2medmYDg,0.000000,0.473782,47.378195
1002,FaHADZARwnY4yvlvpnsfGA,0.000000,0.469587,46.958720
23,na4Th5DrNauOv-c43QQFvA,0.859149,0.459150,46.774136
1003,JAmQCmczUclNUfZjkNdjQA,0.000000,0.456340,45.633988
22,Wxxvi3LZbHNIDwJ-ZimtnA,0.864880,0.445055,45.370426
32,5LNZ67Yw9RD6nf4_UhXOjw,0.859084,0.436633,44.522420
90,VyjyHoBg3KC5BSFRlD0ZPQ,0.782788,0.436636,44.446362
5,qjnJFZtsY_nfRzoL3J_UWQ,0.882702,0.435396,44.422314
1,El4FC8jcawUVgw_0EIcbaQ,0.888830,0.434325,44.321375
